In [1]:
from requests import get
from bs4 import BeautifulSoup

In [2]:
url = 'https://www.imdb.com/search/title/?title_type=tv_series&countries=kr'
response = get(url,headers = {'Accept-Language':'en-US,en;q=0.5'})
# response.text[:500]

In [3]:
html_soup = BeautifulSoup(response.text, 'html.parser')

In [4]:
series_containers = html_soup.find_all('div', class_= 'lister-item mode-advanced')
# print(len(series_containers))

In [5]:
first_series = series_containers[0]
# first_series

In [6]:
first_title = first_series.h3.a.text
first_title

'Pachinko'

In [7]:
first_genre = first_series.p.find('span',class_='genre')
first_genre.text.strip()

'Drama'

In [8]:
second_series = series_containers[1]

In [9]:
second_title = second_series.find('div',class_='lister-item-image float-left')
# second_title

In [10]:
second_genre = second_series.p.find('span',class_='genre')
second_genre.text.strip()

'Action, Drama, Mystery'

In [11]:
first_year = first_series.h3.find('span', class_='lister-item-year text-muted unbold')
first_year = first_year.text
# first_year = int(first_year)

In [12]:
first_imdb = float(first_series.strong.text)
first_imdb

8.4

In [13]:
first_actor = first_series.find('p',class_='')
first_actor = first_actor.text.replace('\n','').strip()
first_actor = first_actor.split(':')
# # len(first_actor)
first_actor=first_actor[-1].split(',')
# first_actor = ','.join([str(actor)for actor in first_actor])
first_actor

['Lee Min-Ho', ' Soji Arai', ' Jin Ha', ' Minha Kim']

In [14]:
first_votes = first_series.find('span',attrs={'name':'nv'})
first_votes = int(first_votes['data-value'])
first_votes

5850

In [15]:
# Lists to store the scraped data
titles = []
years = []
genres = []
imdb_ratings = []
actors = []
votes = []

for container in series_containers:
    if container.find('div', class_='inline-block ratings-imdb-rating') is not None:
        
        title = container.h3.a.text
        titles.append(title)

        year = container.h3.find('span', class_='lister-item-year text-muted unbold')
        year = year.text
        years.append(year)

        genre = container.p.find('span',class_='genre')
        genre = genre.text.strip()
        genres.append(genre)
        
        imdb = float(container.strong.text)
        imdb_ratings.append(imdb)

        actor = container.find('p',class_='')
        actor = actor.text.replace('\n','').strip()
        actor = actor.split(':')
        # len(first_actor)
        actor = actor[-1].split(',')
        actor = ','.join([str(name)for name in actor])
        actors.append(actor)

        vote = container.find('span',attrs={'name':'nv'})['data-value']
        votes.append(int(vote))

In [16]:
import pandas as pd
test_df = pd.DataFrame({'series':titles,'year':years, 'genre':genres,'imdb':imdb_ratings,'actor':actors,'votes':votes})
test_df.head()

,series,year,genre,imdb,actor,votes
0,Pachinko,(2022– ),Drama,8.4,"Lee Min-Ho, Soji Arai, Jin Ha, Minha Kim",5850
1,Squid Game,(2021– ),"Action, Drama, Mystery",8.0,"Lee Jung-jae, Park Hae-soo, Hoyeon, Oh Yeong-su",419478
2,Tomorrow,(2022– ),"Action, Comedy, Drama",7.5,"Kim Hee-seon, Kim Ro Woon, Lee Soo-hyuk, Ji-On...",7744
3,All of Us Are Dead,(2022– ),"Action, Drama, Fantasy",7.5,"Ji-hu Park, Chan-Young Yoon, Yi-Hyun Cho, Park...",42610
4,Miraculous: Tales of Ladybug & Cat Noir,(2015– ),"Animation, Action, Adventure",7.6,"Cristina Valenzuela, Bryce Papenbrook, Keith S...",10751


### Create new dataframe

In [17]:
import numpy as np
from time import sleep
from random import randint
import requests

In [18]:
# Lists to store the scraped data
titles = []
years = []
genres = []
imdb_ratings = []
actors = []
votes = []

# declaring the headers
# headers = {'Accept-Language':'en-US,en;q=0.5'}

# creating an array of values and passing it in the url for dynamic webpages
pages = np.arange(1,2000,50)

# the whole core of the script
for page in pages:
    page = requests.get("https://www.imdb.com/search/title/?title_type=tv_series&countries=kr&start="+str(page)+"&ref_=adv_nxt",headers = {'Accept-Language':'en-US,en;q=0.5'})
    html_soup = BeautifulSoup(page.text, 'html.parser')
    series_containers = html_soup.find_all('div', class_= 'lister-item mode-advanced')
    sleep(randint(2,8))
    for container in series_containers:
        if container.find('div', class_='inline-block ratings-imdb-rating') is not None:
            
            title = container.h3.a.text
            titles.append(title)

            year = container.h3.find('span', class_='lister-item-year text-muted unbold')
            year = year.text
            years.append(year)
            if container.p.find('span',class_='genre') is not None:
                genre = container.p.find('span',class_='genre')
                genre = genre.text.strip()
                genres.append(genre)
            else:
                genre = np.nan
                genres.append(genre)

            imdb = float(container.strong.text)
            imdb_ratings.append(imdb)

            actor = container.find('p',class_='')
            actor = actor.text.replace('\n','').strip()
            actor = actor.split(':')
            # len(first_actor)
            actor = actor[-1].split(',')
            actor = ','.join([str(name)for name in actor])
            actors.append(actor)
            
            vote = container.find('span',attrs={'name':'nv'})['data-value']
            votes.append(int(vote))

        elif container.find('div', class_='inline-block ratings-imdb-rating') is None:
            
            title = container.h3.a.text
            titles.append(title)

            year = container.h3.find('span', class_='lister-item-year text-muted unbold')
            year = year.text
            years.append(year)
            if container.p.find('span',class_='genre') is not None:
                genre = container.p.find('span',class_='genre')
                genre = genre.text.strip()
                genres.append(genre)
            else:
                genre = np.nan
                genres.append(genre)

            imdb = np.nan
            imdb_ratings.append(imdb)

            actor = container.find('p',class_='')
            actor = actor.text.replace('\n','').strip()
            actor = actor.split(':')
            # len(first_actor)
            actor = actor[-1].split(',')
            actor = ','.join([str(name)for name in actor])
            actors.append(actor)
            
            vote = np.nan
            votes.append(vote)

KeyboardInterrupt: 

In [ ]:
test_df_2 = pd.DataFrame({'series':titles,'year':years, 'genre':genres,'imdb':imdb_ratings,'actor':actors,'votes':votes})


In [ ]:
test_df_2.tail()

,series,year,genre,imdb,actor,votes
1995,Rol deo nekseuteu,(2020),Reality-TV,NaN,"Won Seok Heo, Jong In Kim, Su Hyeon Kim, Gyeon...",NaN
1996,Twice Japan,(2017– ),NaN,NaN,,NaN
1997,Queen of Ahyeon,(2007–2008),Drama,NaN,"Se-Jung Kang, Min-sung Kim, Jae-rom Park, Joon...",NaN
1998,Honeykki,(2016– ),Reality-TV,NaN,,NaN
1999,Get It Beauty 2014,(2014),NaN,NaN,,NaN


In [ ]:
test_df_2['year']=test_df_2['year'].fillna(0)

In [ ]:
test_df_2['Release_year']=test_df_2['year'].apply(lambda y: str(y).split("–")[0]).replace('\D+','',regex = True)

In [ ]:
test_df_2.drop(columns=('year'))

,series,genre,imdb,actor,votes,Release_year
0,Pachinko,Drama,8.4,"Lee Min-Ho, Soji Arai, Jin Ha, Minha Kim",5821.0,2022
1,Squid Game,"Action, Drama, Mystery",8.0,"Lee Jung-jae, Park Hae-soo, Hoyeon, Oh Yeong-su",419358.0,2021
2,Tomorrow,"Action, Comedy, Drama",7.4,"Kim Hee-seon, Kim Ro Woon, Lee Soo-hyuk, Ji-On...",7697.0,2022
3,All of Us Are Dead,"Action, Drama, Fantasy",7.5,"Ji-hu Park, Chan-Young Yoon, Yi-Hyun Cho, Park...",42555.0,2022
4,Miraculous: Tales of Ladybug & Cat Noir,"Animation, Action, Adventure",7.6,"Cristina Valenzuela, Bryce Papenbrook, Keith S...",10745.0,2015
...,...,...,...,...,...,...
1995,Rol deo nekseuteu,Reality-TV,NaN,"Won Seok Heo, Jong In Kim, Su Hyeon Kim, Gyeon...",NaN,2020
1996,Twice Japan,NaN,NaN,,NaN,2017
1997,Queen of Ahyeon,Drama,NaN,"Se-Jung Kang, Min-sung Kim, Jae-rom Park, Joon...",NaN,2007
1998,Honeykki,Reality-TV,NaN,,NaN,2016


In [ ]:
year_col = test_df_2.pop('Release_year')
test_df_2.insert(1,'Release_year',year_col)
# test_df_2.drop(columns=('year'),inplace=True)


In [ ]:
# test_df_2[['actor']]
test_df_2['actor']=test_df_2['actor'].replace(r'^\s*$', np.nan, regex=True)


In [ ]:
test_df_2

,series,Release_year,genre,imdb,actor,votes
0,Pachinko,2022,Drama,8.4,"Lee Min-Ho, Soji Arai, Jin Ha, Minha Kim",5821.0
1,Squid Game,2021,"Action, Drama, Mystery",8.0,"Lee Jung-jae, Park Hae-soo, Hoyeon, Oh Yeong-su",419358.0
2,Tomorrow,2022,"Action, Comedy, Drama",7.4,"Kim Hee-seon, Kim Ro Woon, Lee Soo-hyuk, Ji-On...",7697.0
3,All of Us Are Dead,2022,"Action, Drama, Fantasy",7.5,"Ji-hu Park, Chan-Young Yoon, Yi-Hyun Cho, Park...",42555.0
4,Miraculous: Tales of Ladybug & Cat Noir,2015,"Animation, Action, Adventure",7.6,"Cristina Valenzuela, Bryce Papenbrook, Keith S...",10745.0
...,...,...,...,...,...,...
1995,Rol deo nekseuteu,2020,Reality-TV,NaN,"Won Seok Heo, Jong In Kim, Su Hyeon Kim, Gyeon...",NaN
1996,Twice Japan,2017,NaN,NaN,NaN,NaN
1997,Queen of Ahyeon,2007,Drama,NaN,"Se-Jung Kang, Min-sung Kim, Jae-rom Park, Joon...",NaN
1998,Honeykki,2016,Reality-TV,NaN,NaN,NaN


In [ ]:
test_df_2.to_csv('korean_series_imdb.csv')

In [ ]:
test_df_2.groupby(by='genre').count()['series']

genre
Action                       10
Action, Adventure, Comedy    12
Action, Adventure, Crime      1
Action, Adventure, Drama     12
Action, Adventure, Sci-Fi     1
                             ..
Sport                         2
Talk-Show                    24
Thriller                      6
War                           2
Western                       1
Name: series, Length: 219, dtype: int64

In [ ]:
test_df_2['genre'].value_counts()

Drama                       294
Drama, Romance              228
Comedy, Romance             116
Comedy, Drama, Romance      112
Reality-TV                  110
                           ... 
Crime, Fantasy                1
Comedy, History, Romance      1
Comedy, Crime, Mystery        1
Action, Comedy, Mystery       1
Adventure, Reality-TV         1
Name: genre, Length: 219, dtype: int64

In [ ]:
test_df_2.loc[test_df_2['imdb']==test_df_2['imdb'].max(),['series','genre']]

,series,genre
276,Run BTS!,Game-Show
327,BTS: Bon Voyage,Reality-TV
958,More Charming by the Day,"Comedy, Family, Romance"
